## Imports

In [1]:
import pandas as pd

## Data Reading

In [25]:
all_data = pd.read_csv('birds.csv')

In [26]:
print(all_data.head())

   gender  body_mass  beak_length  beak_depth  fin_length bird category
0    male       3750         39.1        18.7         181             A
1  female       3800         39.5        17.4         186             A
2  female       3250         40.3        18.0         195             A
3  female       3500         39.6        17.7         186             A
4  female       3450         36.7        19.3         193             A


In [27]:
print(all_data.iloc[0:2, 1:])

   body_mass  beak_length  beak_depth  fin_length bird category
0       3750         39.1        18.7         181             A
1       3800         39.5        17.4         186             A


## Data Slicing

### Feature Selecting

In [28]:
feature_index = {'gender':0,'body_mass':1, 'beak_length':2, 'beak_depth':3, 'fin_length':4}

In [29]:
feature1, feature2 = 'body_mass', 'beak_length'  # will be taken from the gui
f1, f2 = feature_index[feature1], feature_index[feature2]

### Class Selecting 

In [30]:
class_index = {'A':0, 'B':1, 'C':2}

In [36]:
class1, class2 = 'A', 'C' # will be taken from the gui
b1, b2 = class_index[class1]*50, class_index[class2]*50
e1, e2 = b1+50, b2+50

print(f"Class {class1}: {b1} -> {e1}")
print(f"Class {class2}: {b2} -> {e2}")

Class A: 0 -> 50
Class C: 100 -> 150


## Slicing

In [37]:
C1 = all_data.iloc[b1:e1,[f1, f2]]
C2 = all_data.iloc[b2:e2,[f1, f2]]
data = pd.concat([C1, C2])

In [38]:
print(data.shape)

(100, 2)


In [40]:
print(data.head())

   body_mass  beak_length
0       3750         39.1
1       3800         39.5
2       3250         40.3
3       3500         39.6
4       3450         36.7


In [39]:
print(data.tail())

     body_mass  beak_length
145       4450         50.8
146       3400         50.1
147       4300         49.0
148       3250         51.5
149       3675         49.8


# Preprocessing

# Training

# Testing

# Evaluation